# Classification of Sentinel-1  Time Series

SAR data are not especially good for vegetation classification. However they have the great advantage of being independent of cloud cover. Here we investigate the use of S1 time series over a complete growth period for thematic mapping.

Since we have no ground truth data, we will use the Canada AAFC Annual Crop Inventory, which is also on the GEE archive. In particular the 2017 inventory for an area in southern Saskatchewan. This area consists of large agricultural fields, well-defined crops, and flat terrain (a big advantage for SAR measurement).

Multilook SAR image data are not normally distributed, rather they are gamma distributed. The GEE classifiers might be expected not to work so well, so we will use Tensorflow to program a more flexible neural network classifier.

__First of all we grab a time series for the region of interest over the 2017 growing season (March to October):__

In [1]:
%matplotlib inline
import ee
from auxil.eeWishart import omnibus
ee.Initialize()
poly = ee.Geometry({'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-105.10328600000001, 50.24327999999998], 
                                                                           [-104.66649400000001, 50.24327999999998], 
                                                                           [-104.66649400000001, 50.46604134048255], 
                                                                           [-105.10328600000001, 50.46604134048255], 
                                                                           [-105.10328600000001, 50.24327999999998]]]})
collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
                      .filterBounds(poly) \
                      .filterDate(ee.Date('2017-03-01'), ee.Date('2017-11-01')) \
                      .filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV','VH'])) \
                      .filter(ee.Filter.eq('resolution_meters', 10)) \
                      .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                      .filter(ee.Filter.eq('relativeOrbitNumber_start',5)) \
                      .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))  
collection = collection.sort('system:time_start')

collection.size().getInfo()

19

__Next, we transform the collection into a multiband image with linear intensities, pre-processed with the refined Lee filter:__

In [2]:
from auxil.eeRL import refinedLee
import math

def get_vvvh(image):   
    ''' get 'VV' and 'VH' bands from sentinel-1 imageCollection '''
    return image.select('VV','VH')

timeseries = collection \
            .map(get_vvvh) \
            .map(refinedLee) \
            .toBands() \
            .clip(poly) \
            .float() 

timeseries.bandNames().length().getInfo()

38

__The class lables are conveniently obtained from the GEE archive of the Canadian AAFC Annual Crop Inventory for the year 2017, and we append them as an additional band (band 39):__ 

In [3]:
crop2017 = ee.ImageCollection('AAFC/ACI') \
    .filter(ee.Filter.date('2017-01-01', '2017-12-01')) \
    .first() \
    .clip(poly)\
    .float()
labeled_timeseries = ee.Image.cat(timeseries,crop2017)

labeled_timeseries.bandNames().length().getInfo()

39

__Now export the image to the Google drive (cloud storage would be better, but I don't have a billing account). Note that the export scale is 30m corrresponding to the AAFC/ACI resolution:__

In [ ]:
drexport = ee.batch.Export.image.toDrive(labeled_timeseries,
                  description='driveExportTask', 
                  folder = 'EarthEngineImages',
                  fileNamePrefix='labeled_timeseries',scale=30,maxPixels=1e10)
drexport.start()

__After downloading from the drive to a local directory, we have:__

In [ ]:
!ls -l imagery/regina

__This displays three of the (filtered) VV bands and the last (label) band:__

In [ ]:
%run scripts/dispms -f imagery/regina/labeled_timeseries.tif  -p [21,23,25] \
-F imagery/regina/labeled_timeseries_rl.tif -E 2 -P [57,57,57]

__this displays three of the bmap bands__

In [ ]:
%run scripts/dispms -f imagery/regina/labeled_timeseries.tif -p [52,53,54] -e 2

__Now read the labeled time series into a Numpy array, which we will use to train a Tensorflow NN classifier:__

In [ ]:
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly, GDT_Byte
import numpy as np

gdal.AllRegister()                   
inDataset = gdal.Open('imagery/regina/labeled_timeseries.tif',GA_ReadOnly)
cols = inDataset.RasterXSize
rows = inDataset.RasterYSize    
bands = inDataset.RasterCount
labeled_timeseries = np.zeros((rows*cols,bands))                              
for b in range(bands):
    band = inDataset.GetRasterBand(b+1)
    labeled_timeseries[:,b]=band.ReadAsArray(0,0,cols,rows).ravel()   
labeled_timeseries = np.nan_to_num(labeled_timeseries)   

#multiply the intensities by 100
labeled_timeseries[:,:38] *= 100

# for later
driver = inDataset.GetDriver() 
m = labeled_timeseries.shape[0] 
inDataset = None

labeled_timeseries.shape

__The AAFC/ACI thematic maps have 68 different classes. This code generates a dictionary of class names:__

In [ ]:
classdict = {'0':'Nc'}
filepath = 'imagery/AAFC.txt'
with open(filepath) as fp:
    line = fp.readline()
    key = line[:3].replace('\t','')
    value = line[10:].replace('\t',' ').replace('\n','')
    classdict.update({key:value})
    while line:
        line = fp.readline()
        key = line[:3].replace('\t','')
        value = line[10:].replace('\t','').replace('\n','')
        classdict.update({key:value})
del classdict['']

len(classdict)

__Now we can see which class labels pertain to our region of interest:__

In [ ]:
classnums = np.unique(labeled_timeseries[:,-1])
print(classnums)
classnames = str([classdict[str(int(cn))] for cn in classnums])
classnames

__In order to train the neural network we have to renumber the labels consecutively from 0:__

In [ ]:
i=0
labels = labeled_timeseries[:,-1]
for c in classnums:
    labels = np.where(labels==c,i,labels)
    i += 1  
labels = np.array(labels,dtype=np.uint8) 
n_classes = len(np.unique(labels))
print(np.unique(labels))

__Write the labels as an image to disk and display them:__

In [ ]:
outDataset = driver.Create('imagery/regina/labels.tif',cols,rows,1,GDT_Byte)
outBand = outDataset.GetRasterBand(1)
outBand.WriteArray(np.reshape(labels,(rows,cols)))
outBand.FlushCache()
outDataset = None
%run scripts/dispms -f imagery/regina/labels.tif -c 

__Now we simulate ground truth by taking a random subset of training pixels:__

In [ ]:
# random subset for training
np.random.seed(123)
n = 50000
idx = np.random.permutation(m)[0:n]

# just the intensities (x 100)
Xs1 = labeled_timeseries[idx,:38]

# just the significant changes 
Xs2 = labeled_timeseries[idx,39:-1]

# all training vectors
Xs = labeled_timeseries[idx,:-1]

# one hot encoded class labels
Ls = np.array(labels[idx],dtype=np.int)
ls = np.zeros((n,n_classes))
for i in range(n):
    ls[i,Ls[i]] = 1
print(ls[0:5,:])

__The module class auxil.dnn encapsulates a simple feed forward neural network using tf.keras.models.Sequential(). We use it with two hidden layers, each with 40 neurons, to classifiy the series of intensities (Xs1):__

In [ ]:
%%time
import auxil.dnn as dnn

classifier = dnn.Dnn([40,40],n_classes,learning_rate=0.002)
classifier.train(Xs1,ls,epochs=400)

__The accuracy on the training and validation data is about 86%:__

In [ ]:
classifier.history()

__Now we include the times of significant change in the input vectors (Xs):__

In [ ]:
%%time
import auxil.dnn as dnn

classifier = dnn.Dnn([40,40],n_classes,learning_rate=0.002)
classifier.train(Xs,ls,epochs=400)

In [ ]:
classifier.history()

__We'll use the result to classify (predict) the entire image:__

In [ ]:
cls,probs = classifier.classify(labeled_timeseries[:,0:-1]) 
# for later display:
cls[0]=1
cls[1]=n_classes-1

probs.shape

__Write the thematic map and the class probabilities images to disk:__

In [ ]:
# write the class image to disk
outDataset = driver.Create('imagery/regina/timeseries_class.tif',cols,rows,1,GDT_Byte)
outBand = outDataset.GetRasterBand(1)
outBand.WriteArray(np.reshape(cls,(rows,cols)))
outBand.FlushCache()
outDataset = None
# write the class probabilities to disk
bands = probs.shape[1]
probs = np.byte(probs*255)
outDataset = driver.Create('imagery/regina/timeseries_probs.tif',cols,rows,bands,GDT_Byte)
for b in range(bands):
    outBand = outDataset.GetRasterBand(b+1)
    outBand.WriteArray(np.reshape(probs[:,b],(rows,cols)))
    outBand.FlushCache()
outDataset = None

__Test the classifier with all of the training data (misclassification rate about 14.5%):__

In [ ]:
classifier.test(Xs,ls)

__Compare the classified image with the AAFC/ACI thematic map:__

In [ ]:
%run scripts/dispms -f imagery/regina/timeseries_class.tif -c -F imagery/regina/labels.tif -C

__Looks good, but we can "pretty it up" some more with Probabilistic Label Relaxation (see Chapter 7 in my <a href= "https://www.amazon.com/Analysis-Classification-Change-Detection-Sensing/dp/1138613223/ref=dp_ob_title_bk">textbook</a>):__

In [ ]:
%run scripts/plr imagery/regina/timeseries_probs.tif

%run scripts/dispms -f imagery/regina/timeseries_probs_plr.tif -c  \
                    -F imagery/regina/labels.tif -C -s '/home/mort/Bilder/tmp.png'